In [3]:
import websocket
import requests
import time
import datetime
import json
import pandas as pd

from kucoin.client import Market

In [4]:
KUCOIN_BASE_API_ENDPOINT = "https://api.kucoin.com"
KUCOIN_AUTH_TOKEN_PATH = "/api/v1/bullet-public"
KUCOIN_AUTH_TOKEN_API = KUCOIN_BASE_API_ENDPOINT + KUCOIN_AUTH_TOKEN_PATH
KUCOIN_KLINE_API_PATH = "/api/v1/market/candles"
KUCOIN_KLINE_API = KUCOIN_BASE_API_ENDPOINT + KUCOIN_KLINE_API_PATH
KUCOIN_WSS_BASE_ENDPOINT = "wss://ws-api.kucoin.com/endpoint"
CONNECT_ID = "pjfjweas"

In [9]:
res = requests.post(KUCOIN_BASE_API_ENDPOINT + KUCOIN_AUTH_TOKEN_PATH)

In [28]:
res.headers

{'Date': 'Thu, 18 Nov 2021 19:16:23 GMT', 'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Set-Cookie': 'AWSALB=pv4Oy+BtpiaByxJHJOwXnd55AMT0Swt32/AW3plAyB+U0wYW50IafvjlDqnFvb+6IpMyXo9RQ2xKSXTao9iy6rjUuol0fF3jp6QwhQABxLZmB0bhUBpRNKRuC0bq; Expires=Thu, 25 Nov 2021 19:16:23 GMT; Path=/, AWSALBCORS=pv4Oy+BtpiaByxJHJOwXnd55AMT0Swt32/AW3plAyB+U0wYW50IafvjlDqnFvb+6IpMyXo9RQ2xKSXTao9iy6rjUuol0fF3jp6QwhQABxLZmB0bhUBpRNKRuC0bq; Expires=Thu, 25 Nov 2021 19:16:23 GMT; Path=/; SameSite=None; Secure, x-bullet-token=2neAiuYvAU61ZDXANAGAsiL4-iAExhsBXZxftpOeh_55i3Ysy2q2LEsEWU64mdzUOPusi34M_wGoSf7iNyEWJ1pqalkqTtmhUdb4jIyh6X36chvukOwVMtiYB9J6i9GjsxUuhPw3BlrzazF6ghq4Lw2ZpZvJFTEdVr0f8zcpFe8=.xruRw0LkjUryXRLPZxRBpA==; Max-Age=86400; Expires=Fri, 19-Nov-2021 19:16:23 GMT; Path=/; HttpOnly, __cfruid=a9f2e27e086e839e7d962616b6b67c9b10d5f4ef-1637262983; path=/; domain=.kucoin.com; HttpOnly; Secure; SameSite=None', 'X-Content-Type-Options': 'nosniff', 'X-XSS-Protec

In [5]:
ALLOWED_TIMEFRAME = [
    "1min",
    "3min",
    "5min",
    "15min",
    "30min",
    "1hour",
    "2hour",
    "4hour",
    "6hour",
    "8hour",
    "12hour",
    "1day",
    "1week"
]

In [6]:
client = Market(url=KUCOIN_BASE_API_ENDPOINT)

In [5]:
list_of_symbols_json = client.get_symbol_list()
symbols_list = list(map(lambda x : x['symbol'], list_of_symbols_json))

In [6]:
def extract_all_klines_for_current_symbol(client, symbol, timeframe, start_extraction_timestamp, sleep_duration):
    """Retrieve klines backwards from specified timestamp
    """
    ALLOWED_TIMEFRAME = [
        "1min",
        "3min",
        "5min",
        "15min",
        "30min",
        "1hour",
        "2hour",
        "4hour",
        "6hour",
        "8hour",
        "12hour",
        "1day",
        "1week"
    ]
    klines = []
    current_unix_timestamp = int(start_extraction_timestamp)
    reached_start_of_data = False
    while not reached_start_of_data:
        try:
            retrieved_klines = client.get_kline(symbol, timeframe, endAt=current_unix_timestamp)
            klines += retrieved_klines
            current_unix_timestamp = int(retrieved_klines[-1][0])
            # Max number of rows retrieved is 1500. Hence lesser than 1500 means reached first instance of data
            if len(retrieved_klines) < 1500:
                reached_start_of_data = True
        except:
            data_rows_extracted = len(klines)
            print("Max request reached for requesting {} data (Current timestamp: {}, {} rows extracted so far)! Sleeping for {} seconds....".format(symbol, datetime.datetime.fromtimestamp(current_unix_timestamp), data_rows_extracted, sleep_duration))
            time.sleep(sleep_duration)
            print("Sleep has ended!")
    return klines

In [7]:
start = time.time()
test_klines = extract_all_klines_for_current_symbol(client, symbols_list[0], '1min', time.time(), 10)
end = time.time()

Max request reached for requesting REQ-ETH data (Current timestamp: 2021-10-26 00:04:00, 30000 rows extracted so far)! Sleeping for 10 seconds....
Sleep has ended!
Max request reached for requesting REQ-ETH data (Current timestamp: 2021-10-26 00:04:00, 30000 rows extracted so far)! Sleeping for 10 seconds....
Sleep has ended!
Max request reached for requesting REQ-ETH data (Current timestamp: 2021-10-04 03:04:00, 61500 rows extracted so far)! Sleeping for 10 seconds....
Sleep has ended!
Max request reached for requesting REQ-ETH data (Current timestamp: 2021-05-18 14:04:00, 261000 rows extracted so far)! Sleeping for 10 seconds....
Sleep has ended!
Max request reached for requesting REQ-ETH data (Current timestamp: 2021-04-03 19:04:00, 325500 rows extracted so far)! Sleeping for 10 seconds....
Sleep has ended!
Max request reached for requesting REQ-ETH data (Current timestamp: 2021-03-15 00:04:00, 354000 rows extracted so far)! Sleeping for 10 seconds....
Sleep has ended!
Max request r

In [12]:
def convert_to_dict(single_candlestick_list):
    output = {
        "timestamp": single_candlestick_list[0],
        "o": single_candlestick_list[1],
        "c": single_candlestick_list[2],
        "h": single_candlestick_list[3],
        "l": single_candlestick_list[4],
        "v": single_candlestick_list[5],
        "a": single_candlestick_list[6],
    }
    return output

In [29]:
test_klines_list_dict = list(map(convert_to_dict, test_klines))
test_klines_list_dict = test_klines_list_dict[::-1]

In [30]:
test_df = pd.DataFrame(test_klines_list_dict)

In [58]:
test_df.index[0]

Timestamp('2017-10-29 20:00:00')

In [31]:
test_df["timestamp"] = test_df["timestamp"].map(lambda x : datetime.datetime.fromtimestamp(int(x)))

In [41]:
sample_df = pd.read_parquet("test.parquet")

In [46]:
sample_df.append(pd.DataFrame(test_klines_list_dict[0:10]))

,o,c,h,l,v,a,timestamp
2017-10-29 20:00:00,0.000018,0.000018,0.000018,0.000018,10535,0.18963,NaN
2017-10-29 20:02:00,0.00002,0.00002,0.00002,0.00002,22556,0.45112,NaN
2017-10-29 20:03:00,0.000165,0.00012,0.000165,0.00012,21915.4943,2.78378156,NaN
2017-10-29 20:05:00,0.000163,0.000163,0.000163,0.000163,1710,0.27873,NaN
2017-10-29 20:06:00,0.000163,0.000163,0.000163,0.000163,2559.2733,0.41716155,NaN
...,...,...,...,...,...,...,...
5,0.00016,0.00016,0.00016,0.00016,25,0.004,1509279660
6,0.000152,0.000153,0.000153,0.000152,20000,3.04489341,1509279720
7,0.000152,0.000152,0.000152,0.000152,550,0.0836,1509279780
8,0.000153,0.000153,0.000153,0.000153,50,0.00765,1509281220


In [84]:
klines = client.get_kline('BTC-USDT','1min', endAt=int(time.time()))
klines

[['1636967880',
  '66140',
  '66140',
  '66140',
  '66139.9',
  '0.01120983',
  '741.418008105'],
 ['1636967820',
  '66096',
  '66140',
  '66159.6',
  '66096',
  '4.95051085',
  '327384.102653932'],
 ['1636967760',
  '66146.2',
  '66096',
  '66179.8',
  '66086.4',
  '10.88823522',
  '720016.659391193'],
 ['1636967700',
  '66300',
  '66130.1',
  '66361',
  '66126.4',
  '8.67372455',
  '574798.757360838'],
 ['1636967640',
  '66230.3',
  '66317.9',
  '66317.9',
  '66194.7',
  '8.59911151',
  '569609.126997935'],
 ['1636967580',
  '66184.7',
  '66246',
  '66299.6',
  '66179.3',
  '8.81086536',
  '583693.357763987'],
 ['1636967520',
  '66105.7',
  '66199.3',
  '66280.1',
  '66105.7',
  '17.80396451',
  '1178835.532897236'],
 ['1636967460',
  '66059.5',
  '66105.2',
  '66105.2',
  '66059.4',
  '1.27228959',
  '84071.507913068'],
 ['1636967400',
  '66055',
  '66059.5',
  '66068.3',
  '66035',
  '3.71886438',
  '245630.343602777'],
 ['1636967340',
  '66086.7',
  '66054.9',
  '66086.7',
  '6600

In [86]:
datetime.datetime.fromtimestamp(int(klines[-1][0]))

datetime.datetime(2021, 11, 14, 16, 19)

In [63]:
datetime.datetime.utcfromtimestamp(datetime.datetime.strptime("2001-02-03 10:11:12", "%Y-%m-%d %H:%M:%S"))

TypeError: an integer is required (got type datetime.datetime)

In [56]:
os.path.isfile("test.parquet")

True

In [53]:
os.path.ispath("../data")

AttributeError: module 'ntpath' has no attribute 'ispath'

In [25]:
client.get_symbol_list()

[{'symbol': 'REQ-ETH',
  'name': 'REQ-ETH',
  'baseCurrency': 'REQ',
  'quoteCurrency': 'ETH',
  'feeCurrency': 'ETH',
  'market': 'ALTS',
  'baseMinSize': '1',
  'quoteMinSize': '0.0001',
  'baseMaxSize': '10000000000',
  'quoteMaxSize': '99999999',
  'baseIncrement': '0.0001',
  'quoteIncrement': '0.0000001',
  'priceIncrement': '0.0000001',
  'priceLimitRate': '0.1',
  'isMarginEnabled': False,
  'enableTrading': True},
 {'symbol': 'REQ-BTC',
  'name': 'REQ-BTC',
  'baseCurrency': 'REQ',
  'quoteCurrency': 'BTC',
  'feeCurrency': 'BTC',
  'market': 'BTC',
  'baseMinSize': '1',
  'quoteMinSize': '0.00001',
  'baseMaxSize': '10000000000',
  'quoteMaxSize': '99999999',
  'baseIncrement': '0.0001',
  'quoteIncrement': '0.00000001',
  'priceIncrement': '0.00000001',
  'priceLimitRate': '0.1',
  'isMarginEnabled': False,
  'enableTrading': True},
 {'symbol': 'NULS-ETH',
  'name': 'NULS-ETH',
  'baseCurrency': 'NULS',
  'quoteCurrency': 'ETH',
  'feeCurrency': 'ETH',
  'market': 'ALTS',
  

In [68]:
sample_df = pd.read_parquet("../data/parquet/NULS-ETH_1min.parquet")

In [71]:
client.get_kline("REQ-ETH", "1min", endAt=0)

[['1636995000',
  '0.0000479',
  '0.0000479',
  '0.0000479',
  '0.0000479',
  '9.1585',
  '0.00043869215'],
 ['1636994940', '0.000048', '0.000048', '0.000048', '0.000048', '0', '0'],
 ['1636994880', '0.000048', '0.000048', '0.000048', '0.000048', '0', '0'],
 ['1636994820', '0.000048', '0.000048', '0.000048', '0.000048', '0', '0'],
 ['1636994760', '0.000048', '0.000048', '0.000048', '0.000048', '0', '0'],
 ['1636994700', '0.000048', '0.000048', '0.000048', '0.000048', '0', '0'],
 ['1636994640', '0.000048', '0.000048', '0.000048', '0.000048', '0', '0'],
 ['1636994580', '0.000048', '0.000048', '0.000048', '0.000048', '0', '0'],
 ['1636994520', '0.000048', '0.000048', '0.000048', '0.000048', '0', '0'],
 ['1636994460', '0.000048', '0.000048', '0.000048', '0.000048', '0', '0'],
 ['1636994400', '0.000048', '0.000048', '0.000048', '0.000048', '0', '0'],
 ['1636994340', '0.000048', '0.000048', '0.000048', '0.000048', '0', '0'],
 ['1636994280', '0.000048', '0.000048', '0.000048', '0.000048', '0'

## Web socket testing

In [32]:
temp_token_response = requests.post(KUCOIN_AUTH_TOKEN_API)
temp_token = temp_token_response.json()["data"]["token"]

In [33]:
kucoin_wss_endpoint = KUCOIN_WSS_BASE_ENDPOINT + "?token=" + temp_token + "&connectId=" + CONNECT_ID

In [34]:
ping_json = {
    "id": CONNECT_ID,
    "type": "ping"
}

In [36]:
ws = websocket.create_connection(kucoin_wss_endpoint)
connect_result = ws.recv()
print(connect_result)
ws.send(json.dumps(ping_json))
ping_result = ws.recv()
print(ping_result)

{"id":"pjfjweas","type":"welcome"}
{"id":"pjfjweas","type":"pong"}
